**Table of contents**<a id='toc0_'></a>    
- [Mise en place](#toc1_)    
  - [Récupération des données](#toc1_1_)    
- [🚧 voir cohérence des tags en place](#toc2_)    
- [Nettoyage des données](#toc3_)    
  - [Termes spécifiques](#toc3_1_)    
    - [Tags existants](#toc3_1_1_)    
    - [Langages de programmation de Wikipedia](#toc3_1_2_)    
    - [Ensemble complet des termes](#toc3_1_3_)    
- [XXXXXXXXXXXXXXXXXXXXXXXX](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Mise en place](#toc0_)

In [1]:
import numpy as np
import pandas as pd

# home made functions from the src folder
from src.scrap_and_clean import get_languages
from src.scrap_and_clean import clean_string
from src.scrap_and_clean import exclude_words
from src.scrap_and_clean import rm_ending_punctuation

## <a id='toc1_1_'></a>[Récupération des données](#toc0_)

[Requête SQL data.stackexchange.com ici](https://data.stackexchange.com/stackoverflow/query/1824062/top-50k-relevant-questions)

⇒ compromis fait entre qualité maximale et contraintes (≥ 5 tags)
- pour qualité, score ≥ 10 et > 3 réponses

🚧 >=1 réponse : voir cohérence & qualité

- dates jusqu’à 2009, cohérent
- reste à spectre large pour tout type de question

seules features indispensables & RGPD dans la requête

# <a id='toc2_'></a>[🚧 voir cohérence des tags en place](#toc0_)

In [2]:
df_raw = pd.read_csv("data/2024-03-11 QueryResults.csv")

# <a id='toc3_'></a>[Nettoyage des données](#toc0_)

In [3]:
# remove useless features
df = df_raw.drop(columns=["Score", "AnswerCount", "CreationDate"])

In [4]:
# change Tags string to list
df["Tags"] = df["Tags"].apply(lambda x: x[1:-2].split("><"))

## <a id='toc3_1_'></a>[Termes spécifiques](#toc0_)

🚧 voir utilité => pour le tokenizer ?

🚧 [HTML Parser](https://stackoverflow.com/questions/59660495/python-html-extract-text-into-list) : (cf. [question SoF](https://stackoverflow.com/questions/59660495/python-html-extract-text-into-list))  
[RegEx JL](https://www.notion.so/julmat/RegEx-expressions-r-guli-res-70fbbcb177ee476ba9a5ae011d14fe6f) & [RegEx 101](https://regex101.com/?flavor=python&flags=gm)

### <a id='toc3_1_1_'></a>[Tags existants](#toc0_)

In [5]:
def add_tags(x):
    for e in x:
        tags.add(e)


# use a set to avoid duplicates
tags = set()
df["Tags"].apply(lambda x: add_tags(x))
print(f"Found {len(tags)} existing tags")

Found 22629 existing tags


### <a id='toc3_1_2_'></a>[Langages de programmation de Wikipedia](#toc0_)

Sur StackOverFlow, les noms de langages informatiques dans les dialogues sont fréquents.  
Non seulement ils ont des syntaxes spécifiques (.QL, C++, S, C#...) facilement éliminées par des traitements de caractères spécifiques, mais ils sont également représentatifs du sujet et font souvent partie des tags.

In [6]:
prog_lang = get_languages()
print(f"Found {len(prog_lang)} programming languages on Wikipedia")

Found 692 programming languages on Wikipedia


### <a id='toc3_1_3_'></a>[Ensemble complet des termes](#toc0_)

In [7]:
# concatenate all specific terms
spec_terms = prog_lang.union(tags)
print(f"Total: {len(spec_terms)} specific terms")

Total: 23184 specific terms


Certains termes contiennent des ponctuations finales, à retirer.  
Attention, **seuls les signes ,;:. sont enlevés**, ne correspondant pas à un nom de langage (c#, c++, .qt).

In [8]:
excl_terms = rm_ending_punctuation(list(spec_terms))
print(f"{len(excl_terms)} terms, including 'xcode10.' ? -> {'xcode10.' in excl_terms}")
print(f"{len(excl_terms)} terms, including 'xcode10' ? -> {'xcode10' in excl_terms}")

23184 terms, including 'xcode10.' ? -> False
23184 terms, including 'xcode10' ? -> True


# <a id='toc4_'></a>[XXXXXXXXXXXXXXXXXXXXXXXX](#toc0_)

In [9]:
df["Body"] = df["Body"].apply(clean_string)
df["Title"] = df["Title"].apply(clean_string)

In [10]:
df

,Title,Body,Tags
0,std::shared_mutex::unlock_shared() blocks even...,my team has encountered a deadlock that i susp...,"[c++, windows, multithreading, stl, shared-loc]"
1,"what is the correct output of sizeof(""string"")?","on a microcontroller, in order to avoid loadin...","[c, language-lawyer, sizeof, string-literals, ..."
2,problem loading external scripts like jquery,i'm facing a problem since this morning with w...,"[javascript, html, jquery, google-apps-script,..."
3,does a const char* literal string persistently...,i have functions like the following: and i'm p...,"[c++, constants, constexpr, lifetime, null-ter..."
4,willpopscope is deprecated in flutter,'willpopscope' is deprecated and shouldn't be ...,"[flutter, dart, mobile, cross-platform, deprec..."
...,...,...,...
49995,the activator for bundle is invalid,i'm trying to create a simple plugin in eclips...,"[java, eclipse, eclipse-plugin, osgi, osgi-bundl]"
49996,pre-allocate space for c++ stl queue,i'm writing a radix sort algorithm using queue...,"[c++, performance, memory, stl, queu]"
49997,adjusting the positions of the labels in a uit...,i'm using a uitableviewcell with uitableviewce...,"[iphone, objective-c, cocoa-touch, iphone-sdk-..."
49998,padding on stackpanel?,i am trying to set padding of a stackpanel but...,"[wpf, xaml, layout, padding, stackpane]"


In [11]:
df["Body"][0]

"my team has encountered a deadlock that i suspect is a bug in the windows implementation of srw locks. the code below is a distilled version of real code. here's the summary: main thread acquires exclusive lock main thread creates n children threads each child thread acquires a shared lock spins until all children have acquired a shared lock releases the shared lock main thread releases exclusive lock yes this could be done with std::latch in c++ . that's not the point. this code works most of the time. however roughly in loops it deadlocks. when it deadlocks exactly child successfully acquires a shared lock and n- children are stuck in . on windows this function calls into and blocks in . the behavior is observed when used directly, /, or simply calling functions directly. a raymond chen post asks about this exact behavior, but user error is blamed. this looks like an srw bug to me. it's maybe worth noting that if a child doesn't attempt to latch and calls that this will wake its blo

# 🚧🚧🚧🚧 pourquoi C# passe au travers ??

In [12]:
s = "This is a test string. It contains some <code>code</code> and <img src='img'> and sometimes <other> <unusual> tags, \n newlines \n, UPPERCASE WORDS, suspension dots... lonely numbers 4 654  or 9142 and punctuation ; /*+ and     multiple    spaces  and a+, c++, c#, .ql or even S programming langages."
excluded = ["c++", ".ql", "c#"]
exclude_words(s, excluded)

"This is a test string It contains some <code>code</code> and <img src='img'> and sometimes <other> <unusual> tags newlines  UPPERCASE WORDS suspension dots.. lonely numbers 4 654 or 9142 and punctuation  /*+ and multiple spaces and a+ c# or even S programming langages"

# ☝️  ☝️  ☝️  ☝️  ☝️  ☝️  ☝️  ☝️

In [13]:
import nltk

# exclude specific terms from the tokenizer (C# and others can be tokenized as C and #)

tokens = nltk.word_tokenize("This is a C++ sentence : using coma, m4ny 456 words and 2.5 sentences. C# and .qt are also present.")
tokens


['This',
 'is',
 'a',
 'C++',
 'sentence',
 ':',
 'using',
 'coma',
 ',',
 'm4ny',
 '456',
 'words',
 'and',
 '2.5',
 'sentences',
 '.',
 'C',
 '#',
 'and',
 '.qt',
 'are',
 'also',
 'present',
 '.']